This notebook contains examples of TextClustering model usage for Banking77 datasets.

In [78]:
import pandas as pd
import numpy as np

from omegaconf import OmegaConf

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from utils.data_utils import *
from text_clustering import TextClustering
from utils.evaluation_utils import evaluate_classic_clustering

%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data

To load and split Banking or Demo datasets, you may use functions from ```utils/data_utils.py```. You need to place .csv file with texts (and, possibly, cluester markup) in ```BANKING_PATH``` directory. This directory should contain subdirectory ```embeds/``` with texts embeddings saved as a torch.Tensor of shape ```[n_texts, input_dim]```. 

To add a custom dataset, you need a dataframe containing texts (column 'text') and markup (columns 'cluster') and a tensor of embeddings. Make sure that ```len(dataframe) == embeds.shape[0]```.

In [79]:
BANKING_PATH = "data/banking77/"

data_all, clusters_all, embeds_t5_all = load_banking_data(BANKING_PATH)

NUM_CLUSTERS = 10

base_embeds, _, base_data, base_clusters, _ = sample_banking_clusters(
    dataframe=data_all,
    raw_embeds=embeds_t5_all,
    cluster_num_list=np.linspace(0, NUM_CLUSTERS - 1, NUM_CLUSTERS),
    noise_cluster_num_list=None,
    noise_frac=0
)

INP_DIM = base_embeds.shape[1]

# Classic algorithms

To initialize a classic clustering model, you need to choose dimensionality reduction type (None, 'pca', 'umap', 'tsne' or 'pacmap'), number of components (aka ```feat_dim```), clustering type ('kmeans', 'gmm', 'hdbscan', 'mean_shift' or 'spectral') and its hyper-parameters (```min_samples``` and ```min_cluster_size``` for HDBSCAN; ```bandwidth``` for MeanShift).
Once TextClustering model with the necessary parameters is initialized, it can be fitted and evaluated in the following way.

In [80]:
model = TextClustering(
    n_clusters=NUM_CLUSTERS,
    inp_dim=INP_DIM,
    train_dataset=base_embeds,
    data_frame=base_data,
    feat_dim=10,
    kind="classic clustering",
    dim_reduction_type="umap",
    clustering_type="gmm",
    random_state=42
)

model.fit(base_embeds)
_, clusters = model.transform_and_cluster(base_embeds)

metrics = model.evaluate(use_true_clusters=True)

print(f"Dim red time: {model.times['dim_red']}")
print(f"Clust time: {model.times['clust']}")

silhouette_score: 0.06390000134706497
adjusted_rand_score: 0.6536
adjusted_mutual_info_score: 0.804
average_topic_coherence: 0.7264
Dim red time: 10.43466591835022
Clust time: 0.13676953315734863


To run a series of experiments with grid search and save results in a .csv file, you may use the following function defined in ```evaluation_utils.py```.

In [29]:
results_banking = evaluate_classic_clustering(
    dataset="banking",
    data_path=BANKING_PATH,
    num_clasters=3,
    dim_reduction_type_list=[None, "pca", "pacmap"],
    n_components_list=[None, 10, 50],
    clustering_type_list=["kmeans", "gmm", "hdbscan", "spectral"], # "mean_shift"
    random_state=42,
    min_samples_list=[5, 10],
    min_cluster_size_list=[10, 30, 60],
    bandwidth_list=[1.00, 1.50, 2.00],
    verbose=True,
    save_df=False
)

  0%|                                                                                            | 0/36 [00:00<?, ?it/s]

dim_red: None, n_comp: None, clustering: kmeans


  3%|██▎                                                                                 | 1/36 [00:01<00:48,  1.40s/it]

silhouette_score: 0.10859999805688858
adjusted_rand_score: 0.3873
adjusted_mutual_info_score: 0.5238
average_topic_coherence: 0.7914
--------------------------------------------------------------------------------
dim_red: None, n_comp: None, clustering: gmm


  6%|████▋                                                                               | 2/36 [00:04<01:27,  2.56s/it]

...

silhouette_score: 0.13819999992847443
adjusted_rand_score: 0.9882
adjusted_mutual_info_score: 0.979
average_topic_coherence: 0.8057
min_samples: 10, min_cluster_size: 60



 97%|████████████████████████████████████████████████████████████████████████████████▋  | 35/36 [02:30<00:16, 16.84s/it]

silhouette_score: 0.13819999992847443
adjusted_rand_score: 0.9882
adjusted_mutual_info_score: 0.979
average_topic_coherence: 0.8057
silhouette_score: 0.13819999992847443
adjusted_rand_score: 0.9882
adjusted_mutual_info_score: 0.979
average_topic_coherence: 0.8057
--------------------------------------------------------------------------------
dim_red: pacmap, n_comp: 50, clustering: spectral


100%|███████████████████████████████████████████████████████████████████████████████████| 36/36 [02:39<00:00,  4.43s/it]

silhouette_score: 0.13819999992847443
adjusted_rand_score: 0.9882
adjusted_mutual_info_score: 0.979
average_topic_coherence: 0.8057
--------------------------------------------------------------------------------


In [30]:
results_banking

,Dim reduction,Clustering,Best params,Adjusted Rand,Adjusted Mutual Info,Silhouette Score,Avg topic coherence,"Dim red time, sec","Clustering time, sec"
0,-,kmeans,-,0.3873,0.5238,0.1086,0.7914,0.0000,0.0229
1,-,gmm,-,0.9721,0.9557,0.1381,0.7914,0.0000,0.9545
2,-,hdbscan,min_s = 5 min_cl_s = 30,0.2160,0.4331,-0.0033,0.8559,0.0000,0.5799
3,-,spectral,-,0.8111,0.7803,0.1308,0.5980,0.0000,0.0920
4,"pca, feat_dim = 10",kmeans,-,0.3759,0.5089,0.1087,0.7957,0.0846,0.0240
5,"pca, feat_dim = 10",gmm,-,1.0000,1.0000,0.1384,0.8051,0.1441,0.0350
6,"pca, feat_dim = 10",hdbscan,min_s = 5 min_cl_s = 30,0.6503,0.7048,0.0926,0.7402,0.1320,0.0333
7,"pca, feat_dim = 10",spectral,-,0.8901,0.8470,0.1351,0.7365,0.1306,0.1137
8,"pca, feat_dim = 50",kmeans,-,0.3909,0.5266,0.1071,0.7913,0.4223,0.0242
9,"pca, feat_dim = 50",gmm,-,0.9941,0.9895,0.1382,0.8051,0.0770,0.0399


# Deep clustering

To train a DeepClustering (DEC) model, the same interface can be used. 

In [53]:
model = TextClustering(
    n_clusters=NUM_CLUSTERS,
    inp_dim=INP_DIM,
    train_dataset=base_embeds,
    data_frame=base_data,
    feat_dim=10,
    kind="deep clustering",
    random_state=42
)

model.fit(base_embeds)
_, clusters = model.transform_and_cluster(base_embeds)

metrics = model.evaluate(use_true_clusters=True)

print(f"Dim red time: {model.times['dim_red']}")
print(f"Clust time: {model.times['clust']}")

Phase 1: train embeddings


  0%|          | 0/20 [00:00<?, ?it/s]

Phase 2: train clusters


  0%|          | 0/8 [00:00<?, ?it/s]

silhouette_score: 0.04659999907016754
adjusted_rand_score: 0.3495
adjusted_mutual_info_score: 0.6227
average_topic_coherence: 0.7557
Dim red time: 76.73716926574707
Clust time: 44.125908613204956


# Contrastive Hierarchical Clustering

CoHiClust model can be trained using the same TextClustering wrapper. For this model, you should use OmegaConf configuration file (they are available for Banking and Demo datasets or you can create your own). 

In [57]:
dataset_name = "banking" + str(NUM_CLUSTERS)
cohiclust_cfg = OmegaConf.load(f'cfg/{dataset_name}.yaml')
cohiclust_cfg

{'model': {'name': 'custom', 'inp_dim': 1024, 'out_dim': 128, 'drop_prob': 0.25, 'linear_dims_list': [2048, 2048, 1024]}, 'dataset': {'dataset_name': 'banking10', 'number_classes': 5, 'n_neighb': 5}, 'tree': {'tree_level': 4}, 'simclr': {'temperature': 0.5, 'k': 10, 'feature_dim_projection_head': 128}, 'training': {'epochs': 50, 'batch_size': 64, 'pretraining_epochs': 20, 'start_pruning_epochs': 30, 'leaves_to_prune': 6}}

In [77]:
model = TextClustering(
    n_clusters=NUM_CLUSTERS,
    inp_dim=INP_DIM,
    train_dataset=base_embeds,
    data_frame=base_data,
    cohiclust_cfg=cohiclust_cfg,
    feat_dim=10,
    kind="cohiclust",
    random_state=42
)

model.fit(base_embeds)
_, clusters = model.transform_and_cluster(base_embeds)

metrics = model.evaluate(use_true_clusters=True)

print(f"Dim red time: {model.times['dim_red']}")
print(f"Clust time: {model.times['clust']}")

Computing approximate KNN matrix
Computing approximate KNN matrix
Computing approximate KNN matrix


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

silhouette_score: 0.055399999022483826
adjusted_rand_score: 0.7863
adjusted_mutual_info_score: 0.8578
average_topic_coherence: 0.7775
Dim red time: 0
Clust time: 70.25820064544678
